In [67]:
import glob 
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(7)

DATAPATH = '/Users/wonsuk/projects/data/ecg/raw/2019-11-19'
DATA_LENGTH = 100
TRAIN_RATIO = 0.8

hdf5_files = []
count = 0
for f in glob.glob("{}/*.hd5".format(DATAPATH)):
    count += 1
    if count > DATA_LENGTH:
        break
    hdf5_files.append(f)
        
print(len(hdf5_files))

100


In [12]:
hdf5_files

['/Users/wonsuk/projects/data/ecg/raw/2019-11-19/3844853.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/2969424.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/1493337.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/4199227.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/2280326.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/4841754.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/2075673.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/1639366.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/5197123.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/5432768.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/3121679.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/5674949.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/2910859.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/4691352.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/1202107.hd5',
 '/Users/wonsuk/projects/data/ecg/raw/2019-11-19/276977

In [14]:
ecg_key_string_list = [ 
    "strip_I", 
    "strip_II",
    "strip_III",
    "strip_V1",
    "strip_V2",
    "strip_V3",
    "strip_V4",
    "strip_V5",
    "strip_V6",
    "strip_aVF",
    "strip_aVL",
    "strip_aVR"
]


In [91]:
class ECGDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).int()
        self.target = torch.from_numpy(target).float()
        self.transform = transform
        
    def __getitem__(self, index):
            
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [92]:
x_all = []
y_all = []
for hdf_file in hdf5_files:
    f = h5py.File(hdf_file, 'r')
    y_all.append(f['continuous']['VentricularRate'][0])
    x = np.zeros(shape=(12, 5000))
    for (i, key) in enumerate(ecg_key_string_list):
        x[i][:] = f['ecg_rest'][key][:]
    x_all.append(x)
    
data = ECGDataset(np.asarray(x_all), np.asarray(y_all))

In [93]:
data

In [82]:
train_size = int(TRAIN_RATIO * len(data))
train_size

80

In [84]:
test_size = len(data) - train_size
test_size

20

In [94]:
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

In [1]:
dataloader_train=torch.utils.data.DataLoader(train_dataset,
        batch_size=10, shuffle=True)

for batch_idx, (data, target) in enumerate(dataloader_train):
    print('Batch idx {}, data shape {}, target shape {}'.format(batch_idx, data.shape, target.shape))

NameError: name 'torch' is not defined